# Programming Assignment: Анализ текстов

## Введение
Метод опорных векторов (`Support Vector Machine, SVM`) — один из видов линейных классификаторов. Функционал, который он оптимизирует, направлен на максимизацию ширины разделяющей полосы между классами. Из теории статистического обучения известно, что эта ширина тесно связана с обобщающей способностью алгоритма, а ее максимизация позволяет бороться с переобучением.

Одна из причин популярности линейных методов заключается в том, что они хорошо работают на разреженных данных. Так называются выборки с большим количеством признаков, где на каждом объекте большинство признаков равны нулю. Разреженные данные возникают, например, при работе с текстами. Дело в том, что текст удобно кодировать с помощью "мешка слов" — формируется столько признаков, сколько всего уникальных слов встречается в текстах, и значение каждого признака равно числу вхождений в документ соответствующего слова. Ясно, что общее число различных слов в наборе текстов может достигать десятков тысяч, и при этом лишь небольшая их часть будет встречаться в одном конкретном тексте.

Можно кодировать тексты хитрее, и записывать не количество вхождений слова в текст, а `TF-IDF`. Это показатель, который равен произведению двух чисел: `TF` (`term frequency`) и `IDF` (`inverse document frequency`). Первая равна отношению числа вхождений слова в документ к общей длине документа. Вторая величина зависит от того, в скольки документах выборки встречается это слово. Чем больше таких документов, тем меньше `IDF`. Таким образом, `TF-IDF` будет иметь высокое значение для тех слов, которые много раз встречаются в данном документе, и редко встречаются в остальных.

## Данные
Как мы уже говорили выше, линейные методы часто применяются для решения различных задач анализа текстов. В этом задании мы применим метод опорных векторов для определения того, к какой из тематик относится новость: атеизм или космос.

## Реализация в Scikit-Learn
Для начала вам потребуется загрузить данные. В этом задании мы воспользуемся одним из датасетов, доступных в `scikit-learn`'е — `20 newsgroups`. Для этого нужно воспользоваться модулем `datasets`:

```python
from sklearn import datasets

newsgroups = datasets.fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )
```

После выполнения этого кода массив с текстами будет находиться в поле `newsgroups.data`, номер класса — в поле `newsgroups.target`.

Одна из сложностей работы с текстовыми данными состоит в том, что для них нужно построить числовое представление. Одним из способов нахождения такого представления является вычисление `TF-IDF`. В `Scikit-Learn` это реализовано в классе `sklearn.feature_extraction.text.TfidfVectorizer`. Преобразование обучающей выборки нужно делать с помощью функции `fit_transform`, тестовой — с помощью `transform`.

Реализация `SVM`-классификатора находится в классе `sklearn.svm.SVC`. Веса каждого признака у обученного классификатора хранятся в поле `coef_`. Чтобы понять, какому слову соответствует i-й признак, можно воспользоваться методом `get_feature_names()` у `TfidfVectorizer`:

```python
feature_mapping = vectorizer.get_feature_names()
print(feature_mapping[i])
```

Подбор параметров удобно делать с помощью класса `sklearn.grid_search.GridSearchCV` (При использовании библиотеки `scikit-learn` версии `18.0.1` `sklearn.model_selection.GridSearchCV`). Пример использования:

```python
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(y.size, n_folds=5, shuffle=True, random_state=241)
clf = svm.SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(X, y)
```

При использовании библиотеки `scikit-learn` версии `18.0.1` и выше `KFold` задаётся немного по-другому:

```python
cv = KFold(n_splits=5, shuffle=True, random_state=241)
```

Первым аргументом в `GridSearchCV` передается классификатор, для которого будут подбираться значения параметров, вторым — словарь (`dict`), задающий сетку параметров для перебора. После того, как перебор окончен, можно проанализировать значения качества для всех значений параметров и выбрать наилучший вариант:

```python
for a in gs.grid_scores_:
    # a.mean_validation_score — оценка качества по кросс-валидации
    # a.parameters — значения параметров
```

## Инструкция по выполнению
### Шаг 1:
Загрузите объекты из новостного датасета `20 newsgroups`, относящиеся к категориям "космос" и "атеизм" (инструкция приведена выше). Обратите внимание, что загрузка данных может занять несколько минут

In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.model_selection import KFold, GridSearchCV

In [2]:
newsgroups = fetch_20newsgroups(
                    subset='all', 
                    categories=['alt.atheism', 'sci.space']
             )

### Шаг 2:
Вычислите `TF-IDF`-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить `TF-IDF` по всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко встречаются ситуации, когда признаки объектов тестовой выборки известны на момент обучения, и поэтому можно ими пользоваться при обучении алгоритма.

In [3]:
vectorizer = TfidfVectorizer()

X = vectorizer.fit_transform(newsgroups.data)
y = newsgroups.target

feature_mapping = vectorizer.get_feature_names()

### Шаг 3:
Подберите минимальный лучший параметр `C` из множества $[10^{-5}, 10^{-4}, ... 10^4, 10^5]$ для `SVM` с линейным ядром (`kernel='linear'`) при помощи кросс-валидации по 5 блокам. Укажите параметр `random_state=241` и для `SVM`, и для `KFold`. В качестве меры качества используйте долю верных ответов (`accuracy`).

In [4]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv, n_jobs=-1)
gs.fit(X, y)

GridSearchCV(cv=KFold(n_splits=5, random_state=241, shuffle=True),
       error_score='raise-deprecating',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=241,
  shrinking=True, tol=0.001, verbose=False),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'C': array([1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02,
       1.e+03, 1.e+04, 1.e+05])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=0)

### Шаг 4:
Обучите `SVM` по всей выборке с оптимальным параметром `C`, найденным на предыдущем шаге.

In [5]:
data = {'mean validation score': gs.cv_results_['mean_test_score'], 'parameters': gs.cv_results_['params']}
df = pd.DataFrame(data=data)
df

,mean validation score,parameters
0,0.552632,{'C': 1e-05}
1,0.552632,{'C': 0.0001}
2,0.552632,{'C': 0.001}
3,0.552632,{'C': 0.01}
4,0.950168,{'C': 0.1}
5,0.993281,{'C': 1.0}
6,0.993281,{'C': 10.0}
7,0.993281,{'C': 100.0}
8,0.993281,{'C': 1000.0}
9,0.993281,{'C': 10000.0}


In [6]:
print('Best parameters: ', gs.best_estimator_)

Best parameters:  SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=241,
  shrinking=True, tol=0.001, verbose=False)


In [7]:
clf = gs.best_estimator_
clf.fit(X, y)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=241,
  shrinking=True, tol=0.001, verbose=False)

### Шаг 5:
Найдите 10 слов с наибольшим абсолютным значением веса (веса хранятся в поле `coef_` у `svm.SVC`). Они являются ответом на это задание. Укажите эти слова через запятую или пробел, в нижнем регистре, в лексикографическом порядке.

In [8]:
weights = np.absolute(clf.coef_.toarray())

max_weights = sorted(zip(weights[0], feature_mapping))[-10:]
max_weights.sort(key=lambda x: x[1])
print(max_weights)

[(1.2546899512384038, 'atheism'), (1.2491800073760075, 'atheists'), (1.130612344664901, 'bible'), (1.9203794002294938, 'god'), (1.0970936466401482, 'keith'), (1.2016111817520696, 'moon'), (1.13908083789883, 'religion'), (1.0293069271856938, 'sci'), (1.1801315951388633, 'sky'), (2.6631647884797105, 'space')]


In [9]:
with open("ans.txt", "w") as f:
    for w, c in max_weights[:-1]:
        f.write(c)
        f.write(' ')
    f.write(max_weights[-1][1])

Ответ на каждое задание — текстовый файл, содержащий ответ в первой строчке. Обратите внимание, что отправляемые файлы не должны содержать перевод строки в конце. Данный нюанс является ограничением платформы Coursera. Мы работаем над тем, чтобы убрать это ограничение.